#### Simple Gen AI APP Using Langchain

Just Load All The API Keys

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# langsmith Tracking
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API-KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [4]:
import bs4
from bs4 import BeautifulSoup
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
print(docs)

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAnalyze tr

# divide the documents into chunks
Because, Every LLM model has restrictions with respect to chunk size. If the document context is so huge.


In [7]:
# divide the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
docs = text_splitter.split_documents(docs)
print(docs)

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'), Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started w

In [8]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model = "gemma:2b")


In [9]:
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(docs,embeddings)

In [10]:
print(vectorstore)

In [ ]:
query = "This is where LangSmith can help! LangSmith has LLM-native observability"
result = vectorstore.similarity_search(query)
print(result[0].page_content)

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.


In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4o")

#### to get the meaningful context with respect to question, we use retrieval chain

In [ ]:
# Retrieval chain, Document chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(""" 
Answer the following question based on the context provided:
<context> 
{context}
</context> 
                                                                                                 
""")

document_chain = create_stuff_documents_chain(llm,prompt)
print(document_chain)

You have some text (like a speech or an article).
You break it into small parts using a text splitter.
Now, when the user asks a question like:

"What is the main topic of the speech?"

You want the chatbot to answer using that document.

But how will GPT know the answer?

✅ We must give those document parts to GPT along with the question.

That’s where create_stuff_documents_chain comes in.

In [ ]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":  "This is where LangSmith can help! LangSmith has LLM-native observability",
    "context": [Document(page_content = "This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability")]
     
})

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to 
dynamically select the most related documents and pass those in for a given question.

#### Retreiver

In [ ]:
retriever = vectorstore.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain) 

response = retrieval_chain.invoke({"input": "This is where LangSmith can help! LangSmith has LLM-native observability"})
response["answer"]